In [3]:
import mesa #for MESA
import seaborn as sns #data visualization
import numpy as np
import pandas as pd

#function that will calculate Gini index for the model 
# Gini measures of wealth inequality.
def compute_gini(model:mesa.Model):
  #all agent wealth
  agent_wealths = [agent.wealth for agent in model.schedule.agents]
  x = sorted(agent_wealths)
  N = model.num_agents
  B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
  return 1 + (1 / N) - 2 * B

# Create RectangularRenderer for the grid

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

class RectangularAgentPortraya(mesa.visualization.Porrayal):
  def __init__(self, width, height ,color):
    super().__init__()

Create an agent by creating a new class that extends `mesa.Agent`

In [28]:
class MoneyAgent(mesa.Agent):
  """A money agent with a fixed amount of initial wealth."""
  def __init__(self, unique_id:int, model:mesa.Model, num_other_agents:int, initial_wealth:int):
    #need to be initialized by a unique id and the model (defined later) where it will belong to
    #pass the params to the parent class
    super().__init__(unique_id, model)
    #define our extra variables for our agents as the above just creates an "empty" object
    self.wealth=initial_wealth
    #set the number of all agents to be represented because it will be needed for padding the id with leading zeroes when printed
    self.padding = len(str(num_other_agents))
    #to keep track of steps when no wealth was given
    self.steps_not_given = 0

  """ 
  every agent is expected to have a step() function for the scheduler to call
  """
  def step(self):
    #let's move the agent to a new cell
    self.move()
    #try to give money to another agent in the same cell
    self.give_money()

  """
  every agent can move on the grid they are placed into
  """
  def move(self):
    #get the possible steps, i.e., the surrounding cells
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True, #includes all 8 surrounding squares (other is von Neumann, only left/right,up/down)
      include_center=False #staying in at the same space is not allowed
    )
    #decide the next position
    new_position = self.random.choice(possible_steps)
    #place the agent to the new position
    self.model.grid.move_agent(self, new_position)
  
  """
  every agent can give money to a random other agent in the same cell
  """
  def give_money(self):
    #check if agent is broke
    if self.wealth == 0:
      self.steps_not_given += 1
      return 
    #get the cellmates of the agent
    cellmates = self.model.grid.get_cell_list_contents([self.pos])
    #let's get rid if the agent itself, coz we won't give money for ourselves, doesn't make sense - unnecessary actions
    cellmates.pop(cellmates.index(self))
    #if there is still an agent in the cell
    if(len(cellmates) > 1):
      #chose another agent randomly
      other = self.random.choice(cellmates) 
      # add money to that agent
      other.wealth += 1
      self.wealth -=1
      self.steps_not_given = 0
    else:
      self.steps_not_given += 1

    # print(f"cellmates of agent #{self.unique_id} at position {self.pos}")
    # for i in cellmates:
    #   print(f"{i.unique_id}")


Create the model. The model can be visualized as a grid containing all the agents. The model creates, holds and manages all the agents on the grid. The model evolves in discrete time steps.


In [29]:
class MoneyModel(mesa.Model):
  """A money model with some agents"""
  def __init__(self, num_agents:int, initial_wealth:int, grid_x:int, grid_y:int): #the number of agents is specified by N
    super().__init__()
    #set the number of agents
    self.num_agents = num_agents
    """ 
      Add the scheduler. The scheduler controls the order in which agents are activated, 
      causing the agent to take their defined action. 
      The scheduler is also responsible for advancing the model by one step. Hence,
    """
    # Create scheduler and assign it to the model
    self.schedule = mesa.time.RandomActivation(self)
   
    #create a grid where agents can move
    self.grid = mesa.space.MultiGrid(width=grid_x, height=grid_y, torus=True) #sets grid to be a toroidal, so no edge is there:left continues on the right, and vice versa in all directions

    """Additional variable for batched runs"""
    self.running = True

    #create agents
    for i in range(0, self.num_agents):
      a = MoneyAgent(i, self, num_other_agents = self.num_agents, initial_wealth=initial_wealth)
      #add agent to scheduler
      self.schedule.add(a)
      #add agent to a random cell on the grid
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      self.grid.place_agent(a, (x,y))
      """
      Note: an agent has a pos variable with an (x, y) coordinate tuple. The place_agent method adds the coordinate to the agent automatically.
      """

    """
    Data collector:  model-level variables, agent-level variables, and tables (which are a catch-all for everything else). 
    Model- and agent-level variables are added to the data collector along with a function for collecting them. 
    Model-level collection functions take a model object as an input, while agent-level collection functions take an agent object as an input. 
    Both then return a value computed from the model or each agent at their current state. When the data collector’s collect method is called, 
    with a model object as its argument, it applies each model-level collection function to the model, and stores the results in a dictionary, 
    associating the current value with the current step of the model. Similarly, the method applies each agent-level collection function to 
    each agent currently in the schedule, associating the resulting value with the step of the model, and the agent’s unique_id.
    """
    #on agent level, we collect every agent's wealth as Wealth. And we create a reporter called Gini, that will use the wealth data to compute gini
    #we need to implement the compute_gini function of course
    #we also add a another agent_reporter for wealth tracking and to track the number of steps an agent hasn't given any wealth
    self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, 
            agent_reporters={"Wealth": "wealth", "Steps_not_given": "steps_not_given"} 
        )


  def step(self):
    """ Advance the model by one step """
    self.schedule.step() #this will call the step() function of each agent
    self.datacollector.collect(self) #update data collector at every step
 


Running the model

In [6]:
#instantiate the model
my_money_model = MoneyModel(num_agents=100,grid_x=10,grid_y=10, initial_wealth=1)
#run the model a couple of times
num_steps=100
for i in range(0,num_steps):
  my_money_model.step()
  # print("======================")

visualize the number of agents residing in a cell via seaborn

In [7]:
agent_counts = np.zeros((my_money_model.grid.width, my_money_model.grid.height))
for cell_content, (x,y) in my_money_model.grid.coord_iter():
  agent_count = len(cell_content)
  agent_counts[x][y] = agent_count

#plot using seaborn
g = sns.heatmap(agent_counts, cmap="viridis", annot=True, cbar=False,square=True)
g.figure.set_size_inches(4,4)
g.set(title=f"Number of agents on each cell of the grid after {num_steps} steps");

Get the Gini coefficients

In [8]:
gini = my_money_model.datacollector.get_model_vars_dataframe() #returns the data in a pandas dataframe
# print(gini)
#plot gini
g = sns.lineplot(data=gini)
g.set(title=f"Gini Coefficient over Time for {my_money_model.num_agents} agents",
      ylabel="Gini Coefficient",
      xlabel="Time [Steps]");

Let's get some information out of the model

In [9]:
#list of agent wealths
agent_wealth = [a.wealth for a in my_money_model.schedule.agents]
# print(agent_wealth)
#get min and max for setting xticks
min=np.min(agent_wealth)
max=np.max(agent_wealth)
#create plot using seaborn
g = sns.histplot(agent_wealth, discrete=True)


g.set(title=f"Wealth distribution among {my_money_model.num_agents} agents after {num_steps} steps",
      xlabel="Wealth",
      ylabel="Number of agents",); #semicolon avoids printing out the object info
g.set(xticks=list(range(min,max+1)));

we can get the same info using our collector

In [10]:
agent_wealth = my_money_model.datacollector.get_agent_vars_dataframe()
#agent_wealth.head(n=20)
#get the last step from the dataframe
last_step = agent_wealth.index.get_level_values("Step").max()
# print(last_step)
#get the wealth data for the last step
end_wealth = agent_wealth.xs(key=last_step, level="Step")["Wealth"]
# print(end_wealth)
# Create a histogram of wealth at the last step
g = sns.histplot(end_wealth, discrete=True)
g.set(
    title="Distribution of wealth at the end of simulation",
    xlabel="Wealth",
    ylabel="Number of agents",
);



Plot the wealth of a given agent in the function of time

In [11]:
#get the wealth of agent #14
agent_id = 14
agent_wealth.head()
one_agent_wealth = agent_wealth.xs(key=agent_id, level="AgentID")
print(one_agent_wealth)

#plot
g = sns.lineplot(data=one_agent_wealth, x="Step", y="Wealth")
g.set(title=f"Wealth of agent {agent_id} over time");

      Wealth  Steps_not_given
Step                         
1          1                1
2          0                0
3          0                1
4          0                2
5          0                3
...      ...              ...
96         0                9
97         0               10
98         0               11
99         0               12
100        0               13

[100 rows x 2 columns]


Plot more than one agents wealth change over time

In [12]:
agent_list = [1,14, 44]
# agent_list = agent_wealth.index.get_level_values("AgentID")
# print(agent_list)
#get the wealths of multiple agents over time
multiple_agent_wealth = agent_wealth[agent_wealth.index.get_level_values("AgentID").isin(agent_list)]
print(multiple_agent_wealth)
g = sns.lineplot(data=multiple_agent_wealth, x="Step", y="Wealth", hue="AgentID")
g.set(title=f"Wealth of agents over time");

              Wealth  Steps_not_given
Step AgentID                         
1    44            1                1
     1             2                1
     14            1                1
2    14            0                0
     1             1                0
...              ...              ...
99   14            0               12
     1             0                2
100  1             0                3
     14            0               13
     44            2               13

[300 rows x 2 columns]


# Batch run
We usually won’t run a model only once, but multiple times, with fixed parameters to find the overall distributions the model generates, and with varying parameters to analyze how they drive the model’s outputs and behaviors. Instead of needing to write nested for-loops for each model, Mesa provides a batch_run function which automates it for you.

To make the results a little bit more interesting, we have added a reporter to calculate the number of consecutive time steps an agent hasn’t given any wealth as an agent reporter.

In the following example, we hold the height and width fixed, and vary the number of agents. We tell the batch runner to run 5 instantiations of the model with each number of agents, and to run each for 100 steps.

We want to keep track of
1. the Gini coefficient value at each time step, and
2. the individual agent’s wealth development and steps without giving money.



In [13]:
params = {"grid_x": 10, "grid_y":10, "num_agents":range(5,100,5), "initial_wealth":1}
results = mesa.batch_run(
  MoneyModel,
  parameters=params,
  iterations=5,
  max_steps=100,
  number_processes=1,
  data_collection_period=1,
  display_progress=True,
)

  0%|          | 0/95 [00:00<?, ?it/s]

Convert the results of the batch run to pandas dataframe

In [14]:
results_df = pd.DataFrame(results)
print(results_df.keys())

Index(['RunId', 'iteration', 'Step', 'grid_x', 'grid_y', 'num_agents', 'Gini',
       'AgentID', 'Wealth', 'Steps_not_given'],
      dtype='object')


Let's take a closer look at how the Gini coefficient at the end of each episode changes as we increase the size of the population. For this, we filter our results to only contain the data of one agent (the Gini coefficient will be the same for the entire population at any time) at the 100th step of each episode and then scatter-plot the values for the Gini coefficient over the number of agents. Notice there are five values for each population size since we set iterations=5 when calling the batch run.

In [15]:
results_filtered = results_df[(results_df.AgentID == 0) & (results_df.Step==100)]
results_filtered[["iteration","num_agents","Gini"]].reset_index(drop=True).head()
g = sns.scatterplot(data=results_filtered, x="num_agents", y="Gini")
g.set(
    xlabel="Number of agents",
    ylabel="Gini coefficient",
    title="Gini coefficient vs. number of agents",
);

# Create a point plot with error bars
g = sns.pointplot(data=results_filtered, x="num_agents", y="Steps_not_given", linestyle='none')
g.figure.set_size_inches(8, 4)
g.set(
    xlabel="Number of agents",
    ylabel="Steps no wealth given",
    title="Steps no wealth given vs. number of agents",
);


# Analyzing model reporters: Comparing 5 scenarios
Other insights might be gathered when we compare the Gini coefficient of different scenarios. For example, we can compare the Gini coefficient of a population with 25 agents to the Gini coefficient of a population with 400 agents. While doing this, we increase the number of iterations to 25 to get a better estimate of the Gini coefficient for each population size and get usable error estimations.

In [16]:
params = {"grid_x": 10, "grid_y": 10, "num_agents": [5, 10, 20, 40, 80], "initial_wealth":1}
results_5s = mesa.batch_run(
    MoneyModel,
    parameters=params,
    iterations=100,
    max_steps=120,
    number_processes=1,
    data_collection_period=1,  # Important, otherwise the datacollector will only collect data of the last time step
    display_progress=True,
)

results_5s_df = pd.DataFrame(results_5s)

  0%|          | 0/500 [00:00<?, ?it/s]

In [17]:
# Again filter the results to only contain the data of one agent (the Gini coefficient will be the same for the entire population at any time)
results_5s_df_filtered = results_5s_df[(results_5s_df.AgentID == 0)]
results_5s_df_filtered.head(3)

,RunId,iteration,Step,grid_x,grid_y,num_agents,Gini,AgentID,Wealth,Steps_not_given
3,0,0,1,10,10,5,0.0,0.0,1.0,1.0
9,0,0,2,10,10,5,0.0,0.0,1.0,2.0
15,0,0,3,10,10,5,0.0,0.0,1.0,3.0


In [18]:
# Create a lineplot with error bars
g = sns.lineplot(
    data=results_5s_df,
    x="Step",
    y="Gini",
    hue="num_agents",
    errorbar=("ci", 95),
    palette="tab10",
)
g.figure.set_size_inches(8, 4)
plot_title = "Gini coefficient for different population sizes\n(mean over 100 runs, with 95% confidence interval)"
g.set(title=plot_title, ylabel="Gini coefficient");

From the agents we collected the wealth and the number of consecutive rounds without a transaction. We can compare the 5 different population sizes by plotting the average number of consecutive rounds without a transaction for each population size.

In [19]:
# Calculate the mean of the wealth and the number of consecutive rounds for all agents in each episode
agg_results_df = (
    results_5s_df.groupby(["iteration", "num_agents", "Step"])
    .agg({"Wealth": "mean", "Steps_not_given": "mean"})
    .reset_index()
)
agg_results_df.head(3)

,iteration,num_agents,Step,Wealth,Steps_not_given
0,0,5,0,NaN,NaN
1,0,5,1,1.0,1.0
2,0,5,2,1.0,2.0


In [20]:
# Create a line plot with error bars
g = sns.lineplot(
    data=agg_results_df, x="Step", y="Steps_not_given", hue="num_agents", palette="tab10",errorbar=("ci", 95),
)
g.figure.set_size_inches(8, 4)
g.set(
    title="Average number of consecutive rounds without a transaction for different population sizes\n(mean with 95% confidence interval)",
    ylabel="Consecutive rounds without a transaction",
);

# Visualization (on-the-fly)


In [34]:
from mesa.experimental import JupyterViz
import solara
from matplotlib.figure import Figure

def agent_portrayal(agent):
    #this color and size if for broken agents
    size = 10
    color = "tab:red"
    #if an agent is not broken, we use the following color and size
    if agent.wealth > 0:
        size *= agent.wealth
        color = "tab:blue"
        if agent.wealth > 4:
            color = "tab:green"
            size *=1.5
            if agent.wealth > 6:
                color = "tab:pink"
                size = 100

    
    return { "color": color, "size": size }

def make_histogram(model):
    """let's make another plot that shows the agents and their wealth on a histogram"""
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.schedule.agents]
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)

model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 100,
        "step": 1,
    },
    "initial_wealth": {
        "type": "SliderInt",
        "value":1,
        "label":"Initial wealth",
        "min": 1,
        "max": 5,
        "step" : 1,
    },
    "grid_x": 10,
    "grid_y": 10,
}

page = JupyterViz(
    MoneyModel,
    model_params,
    measures=["Gini", make_histogram],
    name="Money Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text